In [1]:
#Import All dependencies
import pandas as pd
import numpy as np
from keras.layers import *
from keras.models import Model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score

Using TensorFlow backend.
/home/abhijais/.local/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#Load Data
train_data = pd.read_csv('data/train.csv')
eval_data = pd.read_csv('data/test.csv')

print(train_data.info())
print("-"*10)
print(eval_data.info())
print(train_data.head())

#save pId's for eval data
pid = eval_data['PassengerId']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           41

In [3]:
# Check number of null and non null in data
print('Train columns with null values:\n', train_data.isnull().sum())
print("-"*10)

print('Test/Validation columns with null values:\n', train_data.isnull().sum())
print("-"*10)

Train columns with null values:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
----------
Test/Validation columns with null values:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
----------


In [4]:
def preprocess_data(dataset):
    #replace null value of age by median of all non-null age in dataset
    dataset['Age'].fillna(dataset['Age'].median(), inplace = True)
    
    #replace null value of stations by mode of the data (since it is categorical data)
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)
    
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)
    
    
    #Convert categorical data into int
    dataset['Sex'] = pd.Categorical(dataset.Sex).codes
    dataset['Embarked'] = pd.Categorical(dataset.Embarked).codes
    
    
    #Add additional features to data
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset['isAlone'] = 1
    dataset['isAlone'].loc[dataset['FamilySize'] > 1] = 0
    
    dataset['FareBin'] = pd.qcut(dataset['Fare'].astype(int), 4)
    dataset['AgeBin'] = pd.cut(dataset['Age'].astype(int), 5)
    
    #Convert range into numerical
    label = preprocessing.LabelEncoder()
    
    dataset['AgeBin_Code'] = label.fit_transform(dataset['AgeBin'])
    dataset['FareBin_Code'] = label.fit_transform(dataset['FareBin'])
    
    #Drop cols Name, PassengerID, and Cabin since they are most likely not responsible in survival
    drop_cols = ['Name', 'PassengerId', 'Cabin', 'Ticket', 'FareBin', 'AgeBin']
    dataset.drop(drop_cols, axis=1, inplace=True)
    
    return dataset

In [5]:
train_data = preprocess_data(train_data)
eval_data = preprocess_data(eval_data)

# Again check number of null values in data
print('Train columns with null values:\n', train_data.isnull().sum())
print("-"*10)

print('Test/Validation columns with null values:\n', train_data.isnull().sum())
print("-"*10)

Train columns with null values:
 Survived        0
Pclass          0
Sex             0
Age             0
SibSp           0
Parch           0
Fare            0
Embarked        0
FamilySize      0
isAlone         0
AgeBin_Code     0
FareBin_Code    0
dtype: int64
----------
Test/Validation columns with null values:
 Survived        0
Pclass          0
Sex             0
Age             0
SibSp           0
Parch           0
Fare            0
Embarked        0
FamilySize      0
isAlone         0
AgeBin_Code     0
FareBin_Code    0
dtype: int64
----------


/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
print(train_data.shape)
print (train_data.head())

(891, 12)
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  FamilySize  \
0         0       3    1  22.0      1      0   7.2500         2           2   
1         1       1    0  38.0      1      0  71.2833         0           2   
2         1       3    0  26.0      0      0   7.9250         2           1   
3         1       1    0  35.0      1      0  53.1000         2           2   
4         0       3    1  35.0      0      0   8.0500         2           1   

   isAlone  AgeBin_Code  FareBin_Code  
0        0            1             0  
1        0            2             3  
2        1            1             0  
3        0            2             3  
4        1            2             1  


In [8]:
X_train = train_data.loc[:, 'Pclass':]
Y_train = train_data['Survived']
X_train.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,isAlone,AgeBin_Code,FareBin_Code
0,3,1,22.0,1,0,7.2500,2,2,0,1,0
1,1,0,38.0,1,0,71.2833,0,2,0,2,3
2,3,0,26.0,0,0,7.9250,2,1,1,1,0


In [9]:
Y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [11]:
#model 1
from sklearn.svm import SVC
svc = SVC(gamma='auto')

In [12]:
svc.fit(X_train, Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [13]:
#model2
def createModel():
    inp = Input(X_train.shape[1:])
    x = Dense(128, kernel_initializer='normal', activation='relu')(inp)
    #x = Dropout(0.5)(x)
    x = Dense(256, kernel_initializer='normal', activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, kernel_initializer='normal', activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, kernel_initializer='normal', activation='relu')(x)
    #x = Dropout(0.5)(x)
    #     x = Dense(8, kernel_initializer='normal', activation='relu')(x)
    #     x = Dropout(0.5)(x)
    #     x = Dense(4, kernel_initializer='normal', activation='relu')(x)
    #     x = Dropout(0.5)(x)
    x = Dense(1, activation = 'sigmoid')(x)
    model = Model(inputs=inp, outputs=x)
    return model

In [14]:
dnn = createModel()

In [15]:
dnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
dnn.fit(x=X_train.values, y=Y_train.values, validation_split=0.2, epochs=150, batch_size=64)

Train on 712 samples, validate on 179 samples
Epoch 1/150
712/712 [==============================] - 1s 2ms/step - loss: 0.7066 - acc: 0.6110 - val_loss: 0.6681 - val_acc: 0.6425
Epoch 2/150
712/712 [==============================] - 0s 128us/step - loss: 0.7073 - acc: 0.6419 - val_loss: 0.5841 - val_acc: 0.7263
Epoch 3/150
712/712 [==============================] - 0s 140us/step - loss: 0.6807 - acc: 0.6419 - val_loss: 0.5590 - val_acc: 0.7318
Epoch 4/150
712/712 [==============================] - 0s 138us/step - loss: 0.6621 - acc: 0.6882 - val_loss: 0.5515 - val_acc: 0.7095
Epoch 5/150
712/712 [==============================] - 0s 151us/step - loss: 0.6242 - acc: 0.6685 - val_loss: 0.5520 - val_acc: 0.7207
Epoch 6/150
712/712 [==============================] - 0s 137us/step - loss: 0.6140 - acc: 0.6699 - val_loss: 0.5458 - val_acc: 0.7151
Epoch 7/150
712/712 [==============================] - 0s 155us/step - loss: 0.6181 - acc: 0.6756 - val_loss: 0.5422 - val_acc: 0.7374
Epoch 8/150

Epoch 61/150
712/712 [==============================] - 0s 150us/step - loss: 0.4617 - acc: 0.7935 - val_loss: 0.3814 - val_acc: 0.8659
Epoch 62/150
712/712 [==============================] - 0s 149us/step - loss: 0.4517 - acc: 0.8104 - val_loss: 0.3424 - val_acc: 0.8547
Epoch 63/150
712/712 [==============================] - 0s 153us/step - loss: 0.4308 - acc: 0.8230 - val_loss: 0.3561 - val_acc: 0.8380
Epoch 64/150
712/712 [==============================] - 0s 156us/step - loss: 0.4279 - acc: 0.8146 - val_loss: 0.3430 - val_acc: 0.8436
Epoch 65/150
712/712 [==============================] - 0s 160us/step - loss: 0.4493 - acc: 0.7921 - val_loss: 0.3382 - val_acc: 0.8492
Epoch 66/150
712/712 [==============================] - 0s 174us/step - loss: 0.4454 - acc: 0.7949 - val_loss: 0.3804 - val_acc: 0.8547
Epoch 67/150
712/712 [==============================] - 0s 127us/step - loss: 0.4451 - acc: 0.8118 - val_loss: 0.3491 - val_acc: 0.8603
Epoch 68/150
712/712 [==========================

712/712 [==============================] - 0s 146us/step - loss: 0.4216 - acc: 0.8301 - val_loss: 0.3559 - val_acc: 0.8436
Epoch 122/150
712/712 [==============================] - 0s 144us/step - loss: 0.4094 - acc: 0.8272 - val_loss: 0.3668 - val_acc: 0.8547
Epoch 123/150
712/712 [==============================] - 0s 142us/step - loss: 0.4118 - acc: 0.8272 - val_loss: 0.3561 - val_acc: 0.8380
Epoch 124/150
712/712 [==============================] - 0s 132us/step - loss: 0.3919 - acc: 0.8371 - val_loss: 0.3559 - val_acc: 0.8380
Epoch 125/150
712/712 [==============================] - 0s 159us/step - loss: 0.4095 - acc: 0.8258 - val_loss: 0.3568 - val_acc: 0.8380
Epoch 126/150
712/712 [==============================] - 0s 165us/step - loss: 0.4087 - acc: 0.8258 - val_loss: 0.3583 - val_acc: 0.8380
Epoch 127/150
712/712 [==============================] - 0s 149us/step - loss: 0.4021 - acc: 0.8357 - val_loss: 0.3578 - val_acc: 0.8436
Epoch 128/150
712/712 [==============================] 

In [18]:
#model3
parameters = {'bootstrap': True,
              'min_samples_leaf': 3,
              'n_estimators': 200, 
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 6,
              'max_leaf_nodes': None}
RF_model = RandomForestClassifier(**parameters)

In [19]:
#data for model3
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=0)

In [20]:
RF_model.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
RF_predictions = RF_model.predict(X_test)
score = accuracy_score(Y_test ,RF_predictions)
print(score)

0.8379888268156425


In [23]:
pred1 = svc.predict(eval_data)
pred1

array([0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,

In [24]:
pred2 = dnn.predict(eval_data)

In [27]:
pred2 = pred2 > 0.5
pred2 = pred2 * 1
pred2

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
    

In [28]:
pred3 = RF_model.predict(eval_data)

In [29]:
pred3

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [38]:
pred1 = pred1.reshape([418, 1])
pred3 = pred3.reshape([418, 1])
print(pred1.shape)
print(pred2.shape)
print(pred3.shape)

(418, 1)
(418, 1)
(418, 1)


In [47]:
preds = pred1 + pred2 + pred3
preds = preds >= 2
preds = preds * 1
df = pd.DataFrame(preds)
df.head()

,0
0,0
1,0
2,0
3,0
4,0


In [48]:
df['PassengerId'] = pid
df.columns = ['Survived', 'PassengerId']
df.head()

,Survived,PassengerId
0,0,892
1,0,893
2,0,894
3,0,895
4,0,896


In [49]:
df.to_csv('data/best_of_three.csv', index=False)